# Working with Termination Conditions in AutoGen

In this lesson, we'll explore how to control and manage conversations between AI agents using AutoGen's termination conditions. We'll learn about different types of termination conditions and how to combine them for more complex control flow.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.text_analyzer_agent import TextAnalyzerAgent
from autogen.agentchat.contrib.math_agent import MathAgent
from autogen.agentchat.conditions import (
    MaxMessageTermination,
    TextMentionTermination,
    TimeoutTermination
)

## Basic Termination Setup

Let's create a simple scenario where we have a writer agent and an editor agent collaborating on creating and refining poetry:

In [ ]:
# Create our agents
writer = AssistantAgent(
    name="poet",
    system_message="You are a creative poet who writes modern poetry. You accept feedback and improve your work.",
    llm_config={"temperature": 0.7}
)

editor = AssistantAgent(
    name="editor",
    system_message="You are a poetry editor. Review poems and provide constructive feedback. Use 'APPROVED' when satisfied.",
    llm_config={"temperature": 0.3}
)

# Create a simple max message termination
max_messages = MaxMessageTermination(max_messages=6)

## Using Basic Termination Conditions

Let's demonstrate how to use a simple termination condition:

In [ ]:
# Initialize the chat
user_proxy = UserProxyAgent(
    name="user",
    system_message="A poetry enthusiast looking for original poems.",
    code_execution_config=False
)

# Start a conversation with max message termination
chat_messages = user_proxy.initiate_chat(
    writer,
    message="Write a short poem about artificial intelligence",
    termination_condition=max_messages
)

## Combining Termination Conditions

We can create more sophisticated termination logic by combining conditions:

In [ ]:
# Create multiple termination conditions
max_time = TimeoutTermination(timeout=300)  # 5 minutes
approval = TextMentionTermination("APPROVED")
max_rounds = MaxMessageTermination(max_messages=10)

# Combine conditions using OR operator
termination_or = max_rounds | approval  # Stops if either condition is met

# Combine conditions using AND operator
termination_and = max_time & approval  # Stops only if both conditions are met

# Example usage with combined conditions
def poetry_review_session(prompt: str, termination: any):
    return user_proxy.initiate_chat(
        writer,
        message=prompt,
        termination_condition=termination
    )

## Practical Example: Poetry Workshop

Let's create a more complex example where we use termination conditions in a poetry workshop scenario:

In [ ]:
def run_poetry_workshop(theme: str):
    # Create a termination condition that stops when:
    # 1. Editor approves OR
    # 2. Maximum of 8 messages have been exchanged AND 5 minutes have passed
    approval = TextMentionTermination("APPROVED")
    max_messages = MaxMessageTermination(max_messages=8)
    timeout = TimeoutTermination(timeout=300)
    
    termination_condition = approval | (max_messages & timeout)
    
    # Initial prompt
    prompt = f"""Please write a poem about {theme}. 
    The poem should be modern and thought-provoking."""
    
    # Start the workshop session
    chat_result = user_proxy.initiate_chat(
        writer,
        message=prompt,
        termination_condition=termination_condition
    )
    
    return chat_result

# Example usage
workshop_result = run_poetry_workshop("the future of technology")

## Handling Termination Results

We can analyze the results of our terminated conversations:

In [ ]:
def analyze_workshop_results(chat_result):
    # Extract relevant information
    message_count = len(chat_result)
    final_message = chat_result[-1]
    
    # Check termination reason
    if "APPROVED" in final_message.content:
        status = "Approved"
    elif len(chat_result) >= 8:
        status = "Max messages reached"
    else:
        status = "Timeout"
    
    return {
        "status": status,
        "message_count": message_count,
        "final_message": final_message.content
    }

This lesson covered the basics of termination conditions in AutoGen, showing how to:
1. Create basic termination conditions
2. Combine conditions using logical operators
3. Implement practical scenarios with termination conditions
4. Handle and analyze termination results

Remember that termination conditions are essential for controlling the flow of agent conversations and ensuring they conclude appropriately based on your specific requirements.